In [1]:
import torch
import torchvision
from torchvision import datasets,transforms
import matplotlib.pyplot as plt
import numpy as np
import cv2
 
import torch.nn as nn
import torch.nn.functional as F

import tensorboard

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
 
transform = transforms.Compose([
        transforms.ToTensor(), # 转为Tensor
        transforms.Normalize((0.5,), (0.5,)), # 归一化
                             ])
 
train_dataset = torchvision.datasets.MNIST(root='./mnist', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist', train=False, transform=transform, download=True)
 
batch_size = 4
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

 
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, padding=2)
        self.conv2 = nn.Conv2d(6, 16, 5)  
        self.fc1   = nn.Linear(16*5*5, 120)  
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)
 
    def forward(self, x): 
        # print(x.size())    # 4-1-28-28
        
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2)) 
        # print(x.size())    # # 4-6-14-14
        
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        # print("x.size():", x.size())   # 4-16-5-5
        
        x = x.view(x.size()[0], -1)   #展开成一维的
        # print("x.size():", x.size())   # 4-400
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)        
        return x
net = Net()

In [ ]:
#整个测试集上预测
correct = 0
total = 0
 
with torch.no_grad():
    for (images,labels) in testloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs,1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum()

        print('10000张测试集合中的准确率为：', (correct.cpu().numpy()/total * 100))
print(correct)